<a href="https://colab.research.google.com/github/MatSci495/Assignments/blob/main/XRD_Phase_Identification_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pre-Lab: Key Passages from Leeman *et al.* (PRX Energy 2024)

The following excerpts highlight central analytical points made by Leeman *et al.* in their
critique of Szymanski *et al.*.  
Prior to beginning this assigment, read pages 1-6 and 20-22 of Leeman *et al.* and refer back to the guidance provided as you carry out the model fits in this notebook.

---

### 1. Not all goodness-of-fit is equal.
> *“Completely unfitted diffraction features may increase the χ² value by the same amount as slightly incorrect peak shapes, but in practice they represent a far more serious concern, often indicating a missing component of the sample or that the entire model is wrong.”*  
*(Leeman et al., Sec. IV)*

This idea motivates **Problem 2**, where a single-phase model produces reasonable χ² yet leaves large residual peaks that signal the presence of a second known phase.

---

### 2. A discovery claim must be tested against plausible alternatives.
> *“If the wider claim is that a new material has been discovered, it is not enough to show a good match between the proposed model and the data—the fit must be better than that obtained for known materials that are likely to be present.”*  
*(Leeman et al., Sec. IV)*

This principle is central to **Problems 1 and 2**, where you compare “novel” and “known” structural models using χ², residuals, Δχ², F-tests, and AIC.

---

### 3. Missing ordering peaks can be decisive.
> *“The absence of an expected ordering peak, even if that peak is small and makes only a minor numerical difference to the goodness-of-fit, can be fatal for a hypothesis of cation ordering.”*  
*(Leeman et al., Sec. IV)*

This point is illustrated in **Problem 1**, where the “ordered model” predicts a small superlattice reflection that is absent in the synthetic data.

---

### 4. Minor impurity peaks may be statistically visible but epistemically secondary.
> *“A peak of exactly the same size belonging to a minor impurity phase may have almost no relevance to the overall research question, beyond indicating that a small adjustment of the synthetic procedure is needed.”*  
*(Leeman et al., Sec. IV)*

This motivates **Problem 3**, which distinguishes between peaks that matter for the *structural/membership* question and peaks that matter only for *sample quality*.

---

### How to Use These Principles
As you evaluate fits in each problem:
- Look for *structured residuals* and *unfitted peaks*.
- Ask whether each peak matters for the **scientific question**, not just the numerical metric.
- Compare competing models explicitly rather than assuming novelty.
- Reflect on how automated pipelines might conflate “good” χ² with “correct” structure.

These are the same analysis skills that Leeman et al. used to evaluate the A-Lab claims.


# Analysis Set-Up


## Synthetic Rietveld-style Examples Inspired by Leeman et al.

In this notebook, you will work with synthetic powder-diffraction–like data designed to illustrate several points raised by Leeman *et al.* in their critique of Szymanski *et al.*:

- Not all “good fits” are equally informative.
- A claim of a *new material* must be tested **against** plausible known alternatives.
- Small “ordering” peaks (or their absence) can be decisive for cation ordering.
- Peaks from minor impurity/precursor phases can be statistically visible but scientifically secondary.

Treat the pattern as a sum of Gaussian peaks + noise, and evaluate fits using:

- Residual plots
- χ² and reduced χ²
- Δχ² tests between models
- AIC to penalize overly complex models



## Define Functions

In [ ]:
# ============================
# SETUP CELL (RUN FIRST)
# ============================
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.stats import f

# For nicer plots
plt.rcParams['figure.figsize'] = (8, 4)
plt.rcParams['axes.grid'] = True

def gaussian(x, amp, mu, sigma):
    return amp * np.exp(-(x - mu)**2 / (2 * sigma**2))

def multi_gaussian(x, amps, mus, sigmas):
    """
    Sum of multiple Gaussians.
    amps, mus, sigmas are 1D arrays of same length.
    """
    y = np.zeros_like(x)
    for A, m, s in zip(amps, mus, sigmas):
        y += gaussian(x, A, m, s)
    return y

def chi_square(y_obs, y_fit, sigma):
    """
    Compute chi-square given observed data, fit, and assumed constant sigma.
    """
    return np.sum(((y_obs - y_fit) / sigma)**2)

def aic_from_rss(rss, n_params, n_points):
    """
    AIC for least-squares fits with Gaussian errors.
    """
    return 2*n_params + n_points*np.log(rss/n_points)

def plot_fit(x, y_obs, y_fit, title="Fit", ylim=None):
    residuals = y_obs - y_fit
    fig, axes = plt.subplots(2, 1, sharex=True, gridspec_kw={'height_ratios':[3,1]})
    axes[0].plot(x, y_obs, '.', label='Data', markersize=3)
    axes[0].plot(x, y_fit, '-', label='Fit')
    axes[0].set_ylabel("Intensity (a.u.)")
    axes[0].set_title(title)
    axes[0].legend()

    axes[1].plot(x, residuals, '-', label='Residuals')
    axes[1].axhline(0, color='k', linewidth=1)
    axes[1].set_xlabel(r"2θ (deg)")
    axes[1].set_ylabel("Residual")
    if ylim is not None:
        axes[1].set_ylim(ylim)
    plt.tight_layout()
    plt.show()

def f_test(chi2_0, chi2_1, df0, df1):
    """
    F-test comparing model 0 (simpler) vs model 1 (more complex).
    chi2_0, chi2_1: chi-square values
    df0, df1: degrees of freedom (N - k0, N - k1)
    Returns F statistic and p-value.
    """
    num = (chi2_0 - chi2_1) / (df0 - df1)
    den = chi2_1 / df1
    F_stat = num / den
    p_value = 1 - f.cdf(F_stat, df0 - df1, df1)
    return F_stat, p_value


## Generate Synthetic Diffraction Pattern

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Ensure the necessary functions are defined (copied from setup cell for self-containment if needed)
def gaussian(x, amp, mu, sigma):
    return amp * np.exp(-(x - mu)**2 / (2 * sigma**2))

def multi_gaussian(x, amps, mus, sigmas):
    y = np.zeros_like(x)
    for A, m, s in zip(amps, mus, sigmas):
        y += gaussian(x, A, m, s)
    return y

np.random.seed(3) # Use a new seed for this problem for reproducibility

# 2. Define the x-axis range
x_new = np.linspace(10, 50, 2000)

# 3. Define the peak positions (mus)
mus_new = np.array([18.0, 30.0, 36.0, 37.0, 43.0])

# 4. Define the peak amplitudes (amps)
amps_new = np.array([1.3, 0.6, 2.0, 1.9, 3.9])

# 5. Set the peak width
sigma_new = 0.15

# 6. Generate the sum of Gaussian peaks
y_peaks_new = multi_gaussian(x_new, amps_new,
                             mus_new, np.full_like(mus_new, sigma_new))

# 7. Define a small linear background
background_new = 0.02 + 0.0005 * (x_new - x_new.mean())

# 8. Set the noise level
noise_sigma_new = 0.02

# 9. Generate Gaussian noise
noise_new = np.random.normal(0, noise_sigma_new, size=x_new.shape)

# 10. Combine the peaks, background, and noise
y_exp_new = y_peaks_new + background_new + noise_new

# Quick plot of the new

## Plot synthetic diffraction pattern

In [ ]:
plt.plot(x_new, y_exp_new, '.', markersize=2)
plt.xlabel(r"2\theta (deg)")
plt.ylabel("Intensity (a.u.)")
plt.title("Problem: Synthetic Diffraction Pattern")
plt.show()

## Model Information

In [ ]:
import pandas as pd

# Read the CSV file into a DataFrame from GitHub
# IMPORTANT: Use the raw GitHub URL for direct access to the file content.
# Example: 'https://raw.githubusercontent.com/username/repository/main/path/to/xrd%20fit%20values.csv'
df_xrd_fit_values = pd.read_csv('https://raw.githubusercontent.com/MatSci495/Assignments/main/xrd%20fit%20values.csv')

# Display the entire DataFrame
print(df_xrd_fit_values.to_markdown(index=False))